# 构建文本纠错训练数据

标准格式为：
```jsonl

{"id": "A2-0011-1","original_text": "你好！我是张爱文。","correct_text": "你好！我是张爱文。","errors":[]}
{"id": "A2-0023-1","original_text": "下个星期，我跟我朋唷打算去法国玩儿。","correct_text": "下个星期，我跟我朋友打算去法国玩儿。","errors":[{"index": 9,"original_error": "朋唷","type": "spelling_error","correct": "朋友"}]}
{"id": "A2-0023-2","original_text": "我听说，你找到新工作，我很高兴。","correct_text": "我听说，你找到新工作，我很高兴。","errors":[]}
{"id": "A2-0029-1","original_text": "对不气，最近我很忙，所以我不会去妳的。","correct_text": "对不起，最近我很忙，所以我不会去你的。","errors":[{"index": 2,"original_error": "不气","type": "spelling_error","correct": "不起"},{"index": 22,"original_error": "妳的","type": "grammar_error","correct": "你的"}]}
{"id": "A2-0030-1","original_text": "他去昨天了酒吧喝酒。","correct_text": "他昨天去了酒吧喝酒。","errors":[{"index": 1,"original_error": "去昨天了","type": "grammar_error","correct": "昨天去了"}]}
{"id": "A2-0031-1","original_text": "他昨天去了图书馆借了很多书籍。","correct_text": "他昨天去了图书馆借了很多书。","errors":[{"index": 13,"original_error": "书籍","type": "extra_word","correct": "书"}]}
{"id": "A2-0032-1","original_text": "今天我得去趟超市买牛奶。","correct_text": "今天我得去一趟超市买牛奶。","errors":[{"index": 3,"original_error": "去趟","type": "missing_word","correct": "去一趟"}]}
```

## 步骤：
1. 转换 https://huggingface.co/datasets/shibing624/CSC 数据集格式，为标准格式，主要需要新增errors字段的内容；做法：用gpt-4o的api补充错误信息。
2. 使用gpt-4o补充多字、少字错误，以及语法错误；扩充纠错数据集，提升多样性；做法：用few-shot学习的方式，基于self-instruct方法让gpt-4o生成新的纠错case。



## 下载csc的数据

In [1]:
!wget https://huggingface.co/datasets/shibing624/CSC/resolve/main/train.json  -O train.json
!wget https://huggingface.co/datasets/shibing624/CSC/resolve/main/dev.json  -O dev.json
!wget https://huggingface.co/datasets/shibing624/CSC/resolve/main/test.json -O test.json

--2024-08-22 17:55:12--  https://huggingface.co/datasets/shibing624/CSC/resolve/main/train.json
Resolving huggingface.co (huggingface.co)... 202.160.130.145
Connecting to huggingface.co (huggingface.co)|202.160.130.145|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/08/a5/08a550bb5aca91891713cf90cfa7f433f1e9eb5393993a908fca51d7831f42bf/4d947b7c6de671783c9809e90ceeca5142dc716914b06d2f31d0ba3f1eef0dd4?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27train.json%3B+filename%3D%22train.json%22%3B&response-content-type=application%2Fjson&Expires=1724579713&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcyNDU3OTcxM319LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9yZXBvcy8wOC9hNS8wOGE1NTBiYjVhY2E5MTg5MTcxM2NmOTBjZmE3ZjQzM2YxZTllYjUzOTM5OTNhOTA4ZmNhNTFkNzgzMWY0MmJmLzRkOTQ3YjdjNmRlNjcxNzgzYzk4MDllOTBjZWVjYTUxNDJkYzcxNjkxNGIwNmQyZjMxZDBiYTNmMWVlZjBkZDQ%7EcmVzcG9uc

In [2]:
import json
import random
import pandas as pd

def load_json(file_path):
    with open(file_path) as f:
        ds = json.load(f)
    return ds
ds1 = load_json('train.json')
# get random 2000 samples
ds1 = random.sample(ds1, 2000)
print('ds1 size:',len(ds1))
ds2 = load_json('dev.json')
ds2 = random.sample(ds2, 2000)
print('ds2 size:',len(ds2))
ds3 = load_json('test.json')
print('ds3 size:',len(ds3))
ds = ds1 + ds2 + ds3


ds[:3], len(ds)

ds1 size: 2000
ds2 size: 2000
ds3 size: 1100


([{'id': '-',
   'original_text': '今天在瑞典政府主办的一场国际研讨会上，与会代表讨论到每年数千名妇女遭其父兄以所谓荣誉处决方式谋杀的现象，表示唯有藉君提升妇女的社会地位，才能与之对抗。',
   'wrong_ids': [57],
   'correct_text': '今天在瑞典政府主办的一场国际研讨会上，与会代表讨论到每年数千名妇女遭其父兄以所谓荣誉处决方式谋杀的现象，表示唯有藉着提升妇女的社会地位，才能与之对抗。'},
  {'id': '-',
   'original_text': '根据三年前签署的全球暖化问题京都议定书，福有国家保证减少造成温室效应的气体排放量这些气体包括二氧化碳与燃烧石油、煤、天然气等产生的其他副产品。',
   'wrong_ids': [20],
   'correct_text': '根据三年前签署的全球暖化问题京都议定书，富有国家保证减少造成温室效应的气体排放量这些气体包括二氧化碳与燃烧石油、煤、天然气等产生的其他副产品。'},
  {'id': '-',
   'original_text': '据报导，美国战机自二十曰起即在硫磺岛举行军事演习。',
   'wrong_ids': [11],
   'correct_text': '据报导，美国战机自二十日起即在硫磺岛举行军事演习。'}],
 5100)

### 只取有错误的数据，才需要处理，并去重

In [3]:
error_ds = [i for i in ds if len(i['wrong_ids']) > 0]
error_ds[:3], len(error_ds)

([{'id': '-',
   'original_text': '今天在瑞典政府主办的一场国际研讨会上，与会代表讨论到每年数千名妇女遭其父兄以所谓荣誉处决方式谋杀的现象，表示唯有藉君提升妇女的社会地位，才能与之对抗。',
   'wrong_ids': [57],
   'correct_text': '今天在瑞典政府主办的一场国际研讨会上，与会代表讨论到每年数千名妇女遭其父兄以所谓荣誉处决方式谋杀的现象，表示唯有藉着提升妇女的社会地位，才能与之对抗。'},
  {'id': '-',
   'original_text': '根据三年前签署的全球暖化问题京都议定书，福有国家保证减少造成温室效应的气体排放量这些气体包括二氧化碳与燃烧石油、煤、天然气等产生的其他副产品。',
   'wrong_ids': [20],
   'correct_text': '根据三年前签署的全球暖化问题京都议定书，富有国家保证减少造成温室效应的气体排放量这些气体包括二氧化碳与燃烧石油、煤、天然气等产生的其他副产品。'},
  {'id': '-',
   'original_text': '据报导，美国战机自二十曰起即在硫磺岛举行军事演习。',
   'wrong_ids': [11],
   'correct_text': '据报导，美国战机自二十日起即在硫磺岛举行军事演习。'}],
 4515)

In [4]:
input_ds = []
temp = set()
for i in error_ds:
    origin_text = i['original_text']
    if origin_text not in temp:
        # 原始correct_text可能有错误，让gpt-4o来生成
        i['correct_text'] = ''
        i['wrong_ids'] = []
        input_ds.append(i)
    temp.add(origin_text)
    
input_ds[:3], len(input_ds)

([{'id': '-',
   'original_text': '今天在瑞典政府主办的一场国际研讨会上，与会代表讨论到每年数千名妇女遭其父兄以所谓荣誉处决方式谋杀的现象，表示唯有藉君提升妇女的社会地位，才能与之对抗。',
   'wrong_ids': [],
   'correct_text': ''},
  {'id': '-',
   'original_text': '根据三年前签署的全球暖化问题京都议定书，福有国家保证减少造成温室效应的气体排放量这些气体包括二氧化碳与燃烧石油、煤、天然气等产生的其他副产品。',
   'wrong_ids': [],
   'correct_text': ''},
  {'id': '-',
   'original_text': '据报导，美国战机自二十曰起即在硫磺岛举行军事演习。',
   'wrong_ids': [],
   'correct_text': ''}],
 4514)

## gpt-4o api

In [5]:
import os
from openai import AzureOpenAI
from dotenv import load_dotenv
load_dotenv()

AZURE_OPENAI_API_KEY = os.environ.get("AZURE_OPENAI_API_KEY")
print(AZURE_OPENAI_API_KEY)

azclient = AzureOpenAI(
    api_key=AZURE_OPENAI_API_KEY, 
    azure_endpoint="https://completions-gpt-v2.openai.azure.com",
    api_version="2024-02-01",
)
print(azclient)

def request_gpt4o_azure(prompt):
    try:
        r = azclient.chat.completions.create(
            model="gpt-4o",
            messages=[{"role": "user", "content": prompt}],
        )
        return r.choices[0].message.content
    except Exception as e:
        print(e)
        return ""

r = request_gpt4o_azure('为什么我没有被邀请参加父母的婚礼?，一句话回答')
print(r)

297ece6f50c941d39e315bd76c108587
你可能还没有出生吧。


## 步骤1. 转换 https://huggingface.co/datasets/shibing624/CSC 数据集格式，为标准格式，主要需要新增errors字段的内容；做法：用gpt-4o的api补充错误信息。

注意:

- 1.结合gpt模型的推理过程，先输出errors信息，再输出correct_text信息。
- 2.gpt模型对index位置不擅长，所以不需要输出index信息。
- 3.输出的jsonl格式，需要符合标准格式。


In [6]:
def correct_ds(sentence):
    prompt = """
你帮我处理中文纠错训练集，标准纠错样本格式如下（jsonl格式）：

```
{"id": "A2-0011-1","original_text": "你好！我是张爱文。","errors":[],"correct_text": "你好！我是张爱文。"}
{"id": "A2-0023-1","original_text": "下个星期，我跟我朋唷打算去法国玩儿。","errors":[{"original_error": "朋唷","type": "spelling_error","correct": "朋友"}],"correct_text": "下个星期，我跟我朋友打算去法国玩儿。"}
{"id": "A2-0029-1","original_text": "对不气，最近我很忙，所以我不会去妳的。","errors":[{"original_error": "不气","type": "spelling_error","correct": "不起"},{"original_error": "妳的","type": "grammar_error","correct": "你的"}],"correct_text": "对不起，最近我很忙，所以我不会去你的。"}
{"id": "A2-0031-1","original_text": "他昨天去了图书馆借了很多书籍。","errors":[{"original_error": "书籍","type": "extra_word","correct": "书"}],"correct_text": "他昨天去了图书馆借了很多书。"}
{"id": "A2-0032-1","original_text": "今天我得去趟超市买牛奶。","errors":[{"original_error": "去趟","type": "missing_word","correct": "去一趟"}],"correct_text": "今天我得去一趟超市买牛奶。"}
```

说明：
- id：样本id，标识而已，可为空。
- original_text，原始文本：输入的原始文本。
- errors，错误列表：使用 errors 数组来列出每个错误，包括原始错误文本片段（original_error）、错误类型（type）和正确的文本（correct）。这有助于模型更精确地理解错误。
- type，错误类型：定义4种常见的错误类型，例如 spelling_error（拼写错误）、grammar_error（语法错误）、missing_word（缺少单词）、extra_word（多余单词）等。
- correct_text，正确文本：正确的文本，即纠正后的文本。

要求：
1.只按标准纠错样本格式输出处理后的jsonl结果，不需要解释。
2.请确保输出的jsonl格式正确，以便后续处理。
3.errors是个列表，如果有多个错误，就都写入；如果没有错误，就是空列表。

请帮我处理下面的句子：
{sentence}
    """

    try:
        res = request_gpt4o_azure(prompt.replace("{sentence}", sentence))
    except Exception as e:
        print(e)
        res = ""
    
    # 返回结果，处理
    clean_res = ''
    try:
        if res:
            res = res.strip()
            lines = res.split("\n")
            for i in lines:
                if i.startswith("```"):
                    continue
                if i.startswith("{"):
                    clean_res = i
                    break
            clean_res = json.loads(clean_res)
    except Exception as e:
        print(e)
    origin_res = res
    return origin_res, clean_res

# 示例问题和回答
q = """
{'id': '-',
   'original_text': '对不气，最近我很忙，所以我不会去妳的。',
   'wrong_ids': [],
   'correct_text': ''}
"""

# 发送请求并获取评估
origin_res, r = correct_ds(q)
origin_res, r

('```\n{"id": "-","original_text": "对不气，最近我很忙，所以我不会去妳的。","errors":[{"original_error": "不气","type": "spelling_error","correct": "不起"},{"original_error": "妳的","type": "grammar_error","correct": "你的"}],"correct_text": "对不起，最近我很忙，所以我不会去你的。"}\n```',
 {'id': '-',
  'original_text': '对不气，最近我很忙，所以我不会去妳的。',
  'errors': [{'original_error': '不气',
    'type': 'spelling_error',
    'correct': '不起'},
   {'original_error': '妳的', 'type': 'grammar_error', 'correct': '你的'}],
  'correct_text': '对不起，最近我很忙，所以我不会去你的。'})

## 批量处理

In [7]:
from tqdm import tqdm
# 初始化一个空的DataFrame，用于存储预测结果
df = pd.DataFrame(columns=['id', 'original_text', 'errors', 'correct_text'])
output_file_path_temp = 'csc_gpt4o.csv'
output_file_path = 'csc_gpt4o.xlsx'
# 遍历query_list中的每个问题
for idx, q in enumerate(tqdm(input_ds[:], desc="Processing")):
    origin_res, r = correct_ds(str(q))
    if r:
        if idx < 10:
            print(idx, q, origin_res, r)

        new_df = pd.DataFrame([r])
        df = pd.concat([df, new_df], ignore_index=True)

        if idx > 1 and idx % 500 == 0:
            df.to_csv(output_file_path_temp, index=False)

df.to_csv(output_file_path_temp, index=False)
df.to_excel(output_file_path, index=False)

Processing:   0%|          | 1/4514 [00:02<3:42:12,  2.95s/it]

0 {'id': '-', 'original_text': '今天在瑞典政府主办的一场国际研讨会上，与会代表讨论到每年数千名妇女遭其父兄以所谓荣誉处决方式谋杀的现象，表示唯有藉君提升妇女的社会地位，才能与之对抗。', 'wrong_ids': [], 'correct_text': ''} ```
{"id": "-","original_text": "今天在瑞典政府主办的一场国际研讨会上，与会代表讨论到每年数千名妇女遭其父兄以所谓荣誉处决方式谋杀的现象，表示唯有藉君提升妇女的社会地位，才能与之对抗。","errors":[{"original_error": "藉君","type": "spelling_error","correct": "提升"}],"correct_text": "今天在瑞典政府主办的一场国际研讨会上，与会代表讨论到每年数千名妇女遭其父兄以所谓荣誉处决方式谋杀的现象，表示唯有提升妇女的社会地位，才能与之对抗。"}
``` {'id': '-', 'original_text': '今天在瑞典政府主办的一场国际研讨会上，与会代表讨论到每年数千名妇女遭其父兄以所谓荣誉处决方式谋杀的现象，表示唯有藉君提升妇女的社会地位，才能与之对抗。', 'errors': [{'original_error': '藉君', 'type': 'spelling_error', 'correct': '提升'}], 'correct_text': '今天在瑞典政府主办的一场国际研讨会上，与会代表讨论到每年数千名妇女遭其父兄以所谓荣誉处决方式谋杀的现象，表示唯有提升妇女的社会地位，才能与之对抗。'}


Processing:   0%|          | 2/4514 [00:07<4:36:31,  3.68s/it]

1 {'id': '-', 'original_text': '根据三年前签署的全球暖化问题京都议定书，福有国家保证减少造成温室效应的气体排放量这些气体包括二氧化碳与燃烧石油、煤、天然气等产生的其他副产品。', 'wrong_ids': [], 'correct_text': ''} ```
{"id": "-","original_text": "根据三年前签署的全球暖化问题京都议定书，福有国家保证减少造成温室效应的气体排放量这些气体包括二氧化碳与燃烧石油、煤、天然气等产生的其他副产品。","errors":[{"original_error": "福有国家","type": "spelling_error","correct": "富有国家"},{"original_error": "排放量这些气体包括","type": "missing_punctuation","correct": "排放量。这些气体包括"}],"correct_text": "根据三年前签署的全球暖化问题京都议定书，富有国家保证减少造成温室效应的气体排放量。这些气体包括二氧化碳与燃烧石油、煤、天然气等产生的其他副产品。"}
``` {'id': '-', 'original_text': '根据三年前签署的全球暖化问题京都议定书，福有国家保证减少造成温室效应的气体排放量这些气体包括二氧化碳与燃烧石油、煤、天然气等产生的其他副产品。', 'errors': [{'original_error': '福有国家', 'type': 'spelling_error', 'correct': '富有国家'}, {'original_error': '排放量这些气体包括', 'type': 'missing_punctuation', 'correct': '排放量。这些气体包括'}], 'correct_text': '根据三年前签署的全球暖化问题京都议定书，富有国家保证减少造成温室效应的气体排放量。这些气体包括二氧化碳与燃烧石油、煤、天然气等产生的其他副产品。'}


Processing:   0%|          | 3/4514 [00:10<4:36:49,  3.68s/it]

...

Processing:  50%|█████     | 2259/4514 [1:54:46<1:27:50,  2.34s/it]

Expecting property name enclosed in double quotes: line 1 column 2 (char 1)


Processing:  86%|████████▋ | 3896/4514 [3:16:30<28:04,  2.73s/it]  

Expecting property name enclosed in double quotes: line 1 column 2 (char 1)


Processing: 100%|██████████| 4514/4514 [5:50:16<00:00,  4.66s/it]    


In [8]:
df

,id,original_text,errors,correct_text,0
0,-,今天在瑞典政府主办的一场国际研讨会上，与会代表讨论到每年数千名妇女遭其父兄以所谓荣誉处决方式...,"[{'original_error': '藉君', 'type': 'spelling_er...",今天在瑞典政府主办的一场国际研讨会上，与会代表讨论到每年数千名妇女遭其父兄以所谓荣誉处决方式...,NaN
1,-,根据三年前签署的全球暖化问题京都议定书，福有国家保证减少造成温室效应的气体排放量这些气体包括...,"[{'original_error': '福有国家', 'type': 'spelling_...",根据三年前签署的全球暖化问题京都议定书，富有国家保证减少造成温室效应的气体排放量。这些气体包...,NaN
2,-,据报导，美国战机自二十曰起即在硫磺岛举行军事演习。,"[{'original_error': '曰', 'type': 'spelling_err...",据报导，美国战机自二十日起即在硫磺岛举行军事演习。,NaN
3,-,目击者称，以军已把封锁路障除去，不过备关卡检查站仍有坦克派驻，车辆已能来往通过，只是流量很漫。,"[{'original_error': '备', 'type': 'spelling_err...",目击者称，以军已把封锁路障除去，不过各关卡检查站仍有坦克派驻，车辆已能来往通过，只是流量很慢。,NaN
4,-,华盛顿时撮同时报导，华府计划在一年内将驻在南韩首都汉城美军总部的七千多名人员，大部分移往他处。,"[{'original_error': '时撮', 'type': 'spelling_er...",华盛顿时报同时报导，华府计划在一年内将驻在韩国首都首尔美军总部的七千多名人员，大部分移往他处。,NaN
...,...,...,...,...,...
4509,B2-4393-3,整天都有人能检查你在家做甚么，有没有以政府说的去做事。,"[{'original_error': '甚么', 'type': 'spelling_er...",整天都有人能检查你在家做什么，有没有按照政府说的去做事。,NaN
4510,B2-4396-2,但是我觉的学教有焋影机也好，如有发生意外老师们关查比交容易，不一定给家长在家有没有看到，还是...,"[{'original_error': '觉的', 'type': 'spelling_er...",但是我觉得学校有摄像机也好，如有发生意外老师们观察比较容易，不一定给家长在家有没有看到，还是...,NaN
4511,B2-4399-3,然后，因为妈妈或爸爸一直看录音机，所以他们可以帮小孩子解决甚么问题。,"[{'original_error': '看录音机', 'type': 'word_choi...",然后，因为妈妈或爸爸一直听录音机，所以他们可以帮小孩子解决什么问题。,NaN
4512,B2-4399-4,而且老师也很难教师。,"[{'original_error': '教师', 'type': 'grammar_err...",而且老师也很难教。,NaN


In [9]:
import pandas as pd
df1 = pd.read_csv("csc_gpt4o.csv")
df1

,id,original_text,errors,correct_text,0
0,-,今天在瑞典政府主办的一场国际研讨会上，与会代表讨论到每年数千名妇女遭其父兄以所谓荣誉处决方式...,"[{'original_error': '藉君', 'type': 'spelling_er...",今天在瑞典政府主办的一场国际研讨会上，与会代表讨论到每年数千名妇女遭其父兄以所谓荣誉处决方式...,NaN
1,-,根据三年前签署的全球暖化问题京都议定书，福有国家保证减少造成温室效应的气体排放量这些气体包括...,"[{'original_error': '福有国家', 'type': 'spelling_...",根据三年前签署的全球暖化问题京都议定书，富有国家保证减少造成温室效应的气体排放量。这些气体包...,NaN
2,-,据报导，美国战机自二十曰起即在硫磺岛举行军事演习。,"[{'original_error': '曰', 'type': 'spelling_err...",据报导，美国战机自二十日起即在硫磺岛举行军事演习。,NaN
3,-,目击者称，以军已把封锁路障除去，不过备关卡检查站仍有坦克派驻，车辆已能来往通过，只是流量很漫。,"[{'original_error': '备', 'type': 'spelling_err...",目击者称，以军已把封锁路障除去，不过各关卡检查站仍有坦克派驻，车辆已能来往通过，只是流量很慢。,NaN
4,-,华盛顿时撮同时报导，华府计划在一年内将驻在南韩首都汉城美军总部的七千多名人员，大部分移往他处。,"[{'original_error': '时撮', 'type': 'spelling_er...",华盛顿时报同时报导，华府计划在一年内将驻在韩国首都首尔美军总部的七千多名人员，大部分移往他处。,NaN
...,...,...,...,...,...
4509,B2-4393-3,整天都有人能检查你在家做甚么，有没有以政府说的去做事。,"[{'original_error': '甚么', 'type': 'spelling_er...",整天都有人能检查你在家做什么，有没有按照政府说的去做事。,NaN
4510,B2-4396-2,但是我觉的学教有焋影机也好，如有发生意外老师们关查比交容易，不一定给家长在家有没有看到，还是...,"[{'original_error': '觉的', 'type': 'spelling_er...",但是我觉得学校有摄像机也好，如有发生意外老师们观察比较容易，不一定给家长在家有没有看到，还是...,NaN
4511,B2-4399-3,然后，因为妈妈或爸爸一直看录音机，所以他们可以帮小孩子解决甚么问题。,"[{'original_error': '看录音机', 'type': 'word_choi...",然后，因为妈妈或爸爸一直听录音机，所以他们可以帮小孩子解决什么问题。,NaN
4512,B2-4399-4,而且老师也很难教师。,"[{'original_error': '教师', 'type': 'grammar_err...",而且老师也很难教。,NaN


In [10]:
df1 = df1[['id', 'original_text', 'errors', 'correct_text']]
df1 = df1[df1['original_text'].str.len() > 1]
df1 = df1[df1['correct_text'].str.len() > 1]
df = df1
df

,id,original_text,errors,correct_text
0,-,今天在瑞典政府主办的一场国际研讨会上，与会代表讨论到每年数千名妇女遭其父兄以所谓荣誉处决方式...,"[{'original_error': '藉君', 'type': 'spelling_er...",今天在瑞典政府主办的一场国际研讨会上，与会代表讨论到每年数千名妇女遭其父兄以所谓荣誉处决方式...
1,-,根据三年前签署的全球暖化问题京都议定书，福有国家保证减少造成温室效应的气体排放量这些气体包括...,"[{'original_error': '福有国家', 'type': 'spelling_...",根据三年前签署的全球暖化问题京都议定书，富有国家保证减少造成温室效应的气体排放量。这些气体包...
2,-,据报导，美国战机自二十曰起即在硫磺岛举行军事演习。,"[{'original_error': '曰', 'type': 'spelling_err...",据报导，美国战机自二十日起即在硫磺岛举行军事演习。
3,-,目击者称，以军已把封锁路障除去，不过备关卡检查站仍有坦克派驻，车辆已能来往通过，只是流量很漫。,"[{'original_error': '备', 'type': 'spelling_err...",目击者称，以军已把封锁路障除去，不过各关卡检查站仍有坦克派驻，车辆已能来往通过，只是流量很慢。
4,-,华盛顿时撮同时报导，华府计划在一年内将驻在南韩首都汉城美军总部的七千多名人员，大部分移往他处。,"[{'original_error': '时撮', 'type': 'spelling_er...",华盛顿时报同时报导，华府计划在一年内将驻在韩国首都首尔美军总部的七千多名人员，大部分移往他处。
...,...,...,...,...
4509,B2-4393-3,整天都有人能检查你在家做甚么，有没有以政府说的去做事。,"[{'original_error': '甚么', 'type': 'spelling_er...",整天都有人能检查你在家做什么，有没有按照政府说的去做事。
4510,B2-4396-2,但是我觉的学教有焋影机也好，如有发生意外老师们关查比交容易，不一定给家长在家有没有看到，还是...,"[{'original_error': '觉的', 'type': 'spelling_er...",但是我觉得学校有摄像机也好，如有发生意外老师们观察比较容易，不一定给家长在家有没有看到，还是...
4511,B2-4399-3,然后，因为妈妈或爸爸一直看录音机，所以他们可以帮小孩子解决甚么问题。,"[{'original_error': '看录音机', 'type': 'word_choi...",然后，因为妈妈或爸爸一直听录音机，所以他们可以帮小孩子解决什么问题。
4512,B2-4399-4,而且老师也很难教师。,"[{'original_error': '教师', 'type': 'grammar_err...",而且老师也很难教。


In [11]:
output_file_path = 'csc_gpt4o.jsonl'

df.to_json(output_file_path, orient='records', lines=True, force_ascii=False)


print(f"数据已保存到 {output_file_path}")

数据已保存到 csc_gpt4o.jsonl


In [12]:
import json

def convert_to_sharegpt_format(data):
    sharegpt_data = []
    system_prompt = "文本纠错："
    for idx, record in enumerate(data):
        convs = []
        inp = str(record['original_text'])
        err = record['errors']
        output = str(record['correct_text'])
        
        q = system_prompt + '\n\n' + inp
        a = output + '\n\nerrors:' + json.dumps(err, ensure_ascii=False) 
        convs.append([
            {"from": "human", "value": q},
            {"from": "gpt", "value": a}
        ])
        sharegpt_data.append({"conversations": convs})
    return sharegpt_data

   

In [13]:

ds = df.to_dict(orient='records')
print(ds[:3])
# 进行转换
sharegpt_formatted_data = convert_to_sharegpt_format(ds)
sharegpt_formatted_data[:2], len(sharegpt_formatted_data)

[{'id': '-', 'original_text': '今天在瑞典政府主办的一场国际研讨会上，与会代表讨论到每年数千名妇女遭其父兄以所谓荣誉处决方式谋杀的现象，表示唯有藉君提升妇女的社会地位，才能与之对抗。', 'errors': "[{'original_error': '藉君', 'type': 'spelling_error', 'correct': '提升'}]", 'correct_text': '今天在瑞典政府主办的一场国际研讨会上，与会代表讨论到每年数千名妇女遭其父兄以所谓荣誉处决方式谋杀的现象，表示唯有提升妇女的社会地位，才能与之对抗。'}, {'id': '-', 'original_text': '根据三年前签署的全球暖化问题京都议定书，福有国家保证减少造成温室效应的气体排放量这些气体包括二氧化碳与燃烧石油、煤、天然气等产生的其他副产品。', 'errors': "[{'original_error': '福有国家', 'type': 'spelling_error', 'correct': '富有国家'}, {'original_error': '排放量这些气体包括', 'type': 'missing_punctuation', 'correct': '排放量。这些气体包括'}]", 'correct_text': '根据三年前签署的全球暖化问题京都议定书，富有国家保证减少造成温室效应的气体排放量。这些气体包括二氧化碳与燃烧石油、煤、天然气等产生的其他副产品。'}, {'id': '-', 'original_text': '据报导，美国战机自二十曰起即在硫磺岛举行军事演习。', 'errors': "[{'original_error': '曰', 'type': 'spelling_error', 'correct': '日'}]", 'correct_text': '据报导，美国战机自二十日起即在硫磺岛举行军事演习。'}]


([{'conversations': [[{'from': 'human',
      'value': '文本纠错：\n\n今天在瑞典政府主办的一场国际研讨会上，与会代表讨论到每年数千名妇女遭其父兄以所谓荣誉处决方式谋杀的现象，表示唯有藉君提升妇女的社会地位，才能与之对抗。'},
     {'from': 'gpt',
      'value': '今天在瑞典政府主办的一场国际研讨会上，与会代表讨论到每年数千名妇女遭其父兄以所谓荣誉处决方式谋杀的现象，表示唯有提升妇女的社会地位，才能与之对抗。\n\nerrors:"[{\'original_error\': \'藉君\', \'type\': \'spelling_error\', \'correct\': \'提升\'}]"'}]]},
  {'conversations': [[{'from': 'human',
      'value': '文本纠错：\n\n根据三年前签署的全球暖化问题京都议定书，福有国家保证减少造成温室效应的气体排放量这些气体包括二氧化碳与燃烧石油、煤、天然气等产生的其他副产品。'},
     {'from': 'gpt',
      'value': '根据三年前签署的全球暖化问题京都议定书，富有国家保证减少造成温室效应的气体排放量。这些气体包括二氧化碳与燃烧石油、煤、天然气等产生的其他副产品。\n\nerrors:"[{\'original_error\': \'福有国家\', \'type\': \'spelling_error\', \'correct\': \'富有国家\'}, {\'original_error\': \'排放量这些气体包括\', \'type\': \'missing_punctuation\', \'correct\': \'排放量。这些气体包括\'}]"'}]]}],
 4508)

In [14]:
# 输出 JSONL 格式
output_file_path = "sharegpt_formatted_data_csc.jsonl"

with open(output_file_path, 'w', encoding='utf-8') as f:
    for entry in sharegpt_formatted_data:
        f.write(json.dumps(entry, ensure_ascii=False) + '\n')

print(f"数据已保存到 {output_file_path}")

数据已保存到 sharegpt_formatted_data_csc.jsonl


## 步骤2. 使用gpt-4o补充多字、少字错误，以及语法错误；扩充纠错数据集，提升多样性；做法：用few-shot学习的方式，基于self-instruct方法让gpt-4o生成新的纠错case。

In [15]:
!pwd

/Users/xuming/Documents/Codes/pycorrector/scripts


In [20]:

with open("../examples/data/grammar/test_sharegpt.jsonl", 'r') as f:
    data1 = [json.loads(i) for i in f.readlines()]
with open("../examples/data/grammar/train_sharegpt.jsonl", 'r') as f:
    data2 = [json.loads(i) for i in f.readlines()]
    
data = data1 + data2
data[:3], len(data)

([{'conversations': [{'from': 'human',
     'value': '对这个句子语法纠错\n\n2017年《财富》全球论坛自宣布落户广州以来，《财富》杂志将目光聚焦在广州，以财富为媒，向世界展示一个充满底蕴又创新进取的广州。'},
    {'from': 'gpt',
     'value': '自2017年《财富》全球论坛宣布落户广州以来，《财富》杂志将目光聚焦在广州，以财富为媒，向世界展示一个充满底蕴又创新进取的广州。'}]},
  {'conversations': [{'from': 'human',
     'value': '对这个句子语法纠错\n\n它渐渐失去了青苍的颜色，变成嫩黄，变成柔绿，枝条变成细瘦，变成娇弱，好像病了的孩子。'},
    {'from': 'gpt', 'value': '它渐渐失去了青苍的颜色，变成柔绿，变成嫩黄，枝条变成细瘦，变成娇弱，好像病了的孩子。'}]},
  {'conversations': [{'from': 'human',
     'value': '对这个句子语法纠错\n\n可是在从原棉制成纱线的过程，就不像穿着光滑舒适的衣服那样让人愉快了：纱厂工人终日面临着音响，尘埃和湿气的。'},
    {'from': 'gpt',
     'value': '可是在从原棉制成纱线的过程，就不像穿着光滑舒适的衣服那样让人愉快了：纱厂工人终日面临着音响，尘埃和湿气的问题。'}]}],
 1561)

In [21]:
input_sents = [(i['conversations'][0]['value'], i['conversations'][1]['value']) for i in data]
input_sents = [(i.replace("对这个句子语法纠错\n\n", ""),j) for (i,j) in input_sents]
input_sents = [i for i in input_sents if len(i[0]) > 0 and len(i[1]) > 0]
input_sents[:3], len(input_sents)

([('2017年《财富》全球论坛自宣布落户广州以来，《财富》杂志将目光聚焦在广州，以财富为媒，向世界展示一个充满底蕴又创新进取的广州。',
   '自2017年《财富》全球论坛宣布落户广州以来，《财富》杂志将目光聚焦在广州，以财富为媒，向世界展示一个充满底蕴又创新进取的广州。'),
  ('它渐渐失去了青苍的颜色，变成嫩黄，变成柔绿，枝条变成细瘦，变成娇弱，好像病了的孩子。',
   '它渐渐失去了青苍的颜色，变成柔绿，变成嫩黄，枝条变成细瘦，变成娇弱，好像病了的孩子。'),
  ('可是在从原棉制成纱线的过程，就不像穿着光滑舒适的衣服那样让人愉快了：纱厂工人终日面临着音响，尘埃和湿气的。',
   '可是在从原棉制成纱线的过程，就不像穿着光滑舒适的衣服那样让人愉快了：纱厂工人终日面临着音响，尘埃和湿气的问题。')],
 1561)

In [22]:
input_grammar_ds = [{"id": f'grammar-{id}', "original_text": sent[0], "wrong_ids": [], "correct_text": ""} for id,sent in enumerate(input_sents)]
input_grammar_ds[:3], len(input_grammar_ds)

([{'id': 'grammar-0',
   'original_text': '2017年《财富》全球论坛自宣布落户广州以来，《财富》杂志将目光聚焦在广州，以财富为媒，向世界展示一个充满底蕴又创新进取的广州。',
   'wrong_ids': [],
   'correct_text': ''},
  {'id': 'grammar-1',
   'original_text': '它渐渐失去了青苍的颜色，变成嫩黄，变成柔绿，枝条变成细瘦，变成娇弱，好像病了的孩子。',
   'wrong_ids': [],
   'correct_text': ''},
  {'id': 'grammar-2',
   'original_text': '可是在从原棉制成纱线的过程，就不像穿着光滑舒适的衣服那样让人愉快了：纱厂工人终日面临着音响，尘埃和湿气的。',
   'wrong_ids': [],
   'correct_text': ''}],
 1561)

In [23]:
# 示例问题和回答
q = """
{'id': '-',
   'original_text': '可是在从原棉制成纱线的过程，就不像穿着光滑舒适的衣服那样让人愉快了：纱厂工人终日面临着音响，尘埃和湿气的。',
   'wrong_ids': [],
   'correct_text': ''}
"""

# 发送请求并获取评估
origin_res, r = correct_ds(q)
origin_res, r

('```\n{"id": "-","original_text": "可是在从原棉制成纱线的过程，就不像穿着光滑舒适的衣服那样让人愉快了：纱厂工人终日面临着音响，尘埃和湿气的。","errors":[{"original_error": "音响","type": "spelling_error","correct": "音响（应为噪音）"}],"correct_text": "可是在从原棉制成纱线的过程，就不像穿着光滑舒适的衣服那样让人愉快了：纱厂工人终日面临着噪音，尘埃和湿气的。"}\n```',
 {'id': '-',
  'original_text': '可是在从原棉制成纱线的过程，就不像穿着光滑舒适的衣服那样让人愉快了：纱厂工人终日面临着音响，尘埃和湿气的。',
  'errors': [{'original_error': '音响',
    'type': 'spelling_error',
    'correct': '音响（应为噪音）'}],
  'correct_text': '可是在从原棉制成纱线的过程，就不像穿着光滑舒适的衣服那样让人愉快了：纱厂工人终日面临着噪音，尘埃和湿气的。'})

In [24]:
for i in input_grammar_ds[:2]:
    print(i)
    q = str(i)
    origin_res, r = correct_ds(q)
    print(origin_res, r)



{'id': 'grammar-0', 'original_text': '2017年《财富》全球论坛自宣布落户广州以来，《财富》杂志将目光聚焦在广州，以财富为媒，向世界展示一个充满底蕴又创新进取的广州。', 'wrong_ids': [], 'correct_text': ''}
```
{"id": "grammar-0", "original_text": "2017年《财富》全球论坛自宣布落户广州以来，《财富》杂志将目光聚焦在广州，以财富为媒，向世界展示一个充满底蕴又创新进取的广州。", "errors": [], "correct_text": "2017年《财富》全球论坛自宣布落户广州以来，《财富》杂志将目光聚焦在广州，以财富为媒，向世界展示一个充满底蕴又创新进取的广州。"}
``` {'id': 'grammar-0', 'original_text': '2017年《财富》全球论坛自宣布落户广州以来，《财富》杂志将目光聚焦在广州，以财富为媒，向世界展示一个充满底蕴又创新进取的广州。', 'errors': [], 'correct_text': '2017年《财富》全球论坛自宣布落户广州以来，《财富》杂志将目光聚焦在广州，以财富为媒，向世界展示一个充满底蕴又创新进取的广州。'}
{'id': 'grammar-1', 'original_text': '它渐渐失去了青苍的颜色，变成嫩黄，变成柔绿，枝条变成细瘦，变成娇弱，好像病了的孩子。', 'wrong_ids': [], 'correct_text': ''}
```json
{"id": "grammar-1","original_text": "它渐渐失去了青苍的颜色，变成嫩黄，变成柔绿，枝条变成细瘦，变成娇弱，好像病了的孩子。","errors":[],"correct_text":"它渐渐失去了青苍的颜色，变成嫩黄，变成柔绿，枝条变成细瘦，变成娇弱，好像病了的孩子。"}
``` {'id': 'grammar-1', 'original_text': '它渐渐失去了青苍的颜色，变成嫩黄，变成柔绿，枝条变成细瘦，变成娇弱，好像病了的孩子。', 'errors': [], 'correct_text': '它渐渐失去了青苍的颜色，变成嫩黄，变成柔绿，枝条变成细瘦，变成娇弱，好像病了的孩子。'}

## 批量执行

In [29]:

from tqdm import tqdm
# 初始化一个空的DataFrame，用于存储预测结果
df = pd.DataFrame(columns=['id', 'original_text', 'errors', 'correct_text'])
output_file_path_temp = 'grammar_gpt4o.csv'
output_file_path = 'grammar_gpt4o.xlsx'
output_file_path_jsonl = 'grammar_gpt4o.jsonl'
# 遍历query_list中的每个问题
for idx, q in enumerate(tqdm(input_grammar_ds[:], desc="Processing")):
    origin_res, r = correct_ds(str(q))
    if r:
        if idx < 10:
            print(idx, q, origin_res, r)

        new_df = pd.DataFrame([r])
        df = pd.concat([df, new_df], ignore_index=True)

        if idx > 1 and idx % 10 == 0:
            df.to_csv(output_file_path_temp, index=False)

df.to_csv(output_file_path_temp, index=False)
df.to_excel(output_file_path, index=False)
df.to_json(output_file_path_jsonl, orient='records', lines=True, force_ascii=False)

Processing:   0%|          | 1/1561 [00:04<2:03:21,  4.74s/it]

0 {'id': 'grammar-0', 'original_text': '2017年《财富》全球论坛自宣布落户广州以来，《财富》杂志将目光聚焦在广州，以财富为媒，向世界展示一个充满底蕴又创新进取的广州。', 'wrong_ids': [], 'correct_text': ''} ```
{"id": "grammar-0", "original_text": "2017年《财富》全球论坛自宣布落户广州以来，《财富》杂志将目光聚焦在广州，以财富为媒，向世界展示一个充满底蕴又创新进取的广州。", "errors": [], "correct_text": "2017年《财富》全球论坛自宣布落户广州以来，《财富》杂志将目光聚焦在广州，以财富为媒，向世界展示一个充满底蕴又创新进取的广州。"}
``` {'id': 'grammar-0', 'original_text': '2017年《财富》全球论坛自宣布落户广州以来，《财富》杂志将目光聚焦在广州，以财富为媒，向世界展示一个充满底蕴又创新进取的广州。', 'errors': [], 'correct_text': '2017年《财富》全球论坛自宣布落户广州以来，《财富》杂志将目光聚焦在广州，以财富为媒，向世界展示一个充满底蕴又创新进取的广州。'}


Processing:   0%|          | 2/1561 [00:07<1:32:39,  3.57s/it]

1 {'id': 'grammar-1', 'original_text': '它渐渐失去了青苍的颜色，变成嫩黄，变成柔绿，枝条变成细瘦，变成娇弱，好像病了的孩子。', 'wrong_ids': [], 'correct_text': ''} ```jsonl
{"id": "grammar-1","original_text": "它渐渐失去了青苍的颜色，变成嫩黄，变成柔绿，枝条变成细瘦，变成娇弱，好像病了的孩子。","errors":[],"correct_text": "它渐渐失去了青苍的颜色，变成嫩黄，变成柔绿，枝条变成细瘦，变成娇弱，好像病了的孩子。"}
``` {'id': 'grammar-1', 'original_text': '它渐渐失去了青苍的颜色，变成嫩黄，变成柔绿，枝条变成细瘦，变成娇弱，好像病了的孩子。', 'errors': [], 'correct_text': '它渐渐失去了青苍的颜色，变成嫩黄，变成柔绿，枝条变成细瘦，变成娇弱，好像病了的孩子。'}


Processing:   0%|          | 3/1561 [00:10<1:31:52,  3.54s/it]

...

Processing:   1%|          | 10/1561 [00:30<1:11:30,  2.77s/it]

9 {'id': 'grammar-9', 'original_text': '莫言的作品充满着“怀乡”以及“怨乡”的复杂情感，建构了独特的主观感觉世界，带有明显的“先锋”色彩，被归类为“寻根文学”作家。', 'wrong_ids': [], 'correct_text': ''} ```
{"id": "grammar-9", "original_text": "莫言的作品充满着“怀乡”以及“怨乡”的复杂情感，建构了独特的主观感觉世界，带有明显的“先锋”色彩，被归类为“寻根文学”作家。", "errors": [], "correct_text": "莫言的作品充满着“怀乡”以及“怨乡”的复杂情感，建构了独特的主观感觉世界，带有明显的“先锋”色彩，被归类为“寻根文学”作家。"}
``` {'id': 'grammar-9', 'original_text': '莫言的作品充满着“怀乡”以及“怨乡”的复杂情感，建构了独特的主观感觉世界，带有明显的“先锋”色彩，被归类为“寻根文学”作家。', 'errors': [], 'correct_text': '莫言的作品充满着“怀乡”以及“怨乡”的复杂情感，建构了独特的主观感觉世界，带有明显的“先锋”色彩，被归类为“寻根文学”作家。'}


Processing:  46%|████▌     | 719/1561 [43:02<43:11,  3.08s/it]  

Extra data: line 1 column 223 (char 222)


Processing: 100%|██████████| 1561/1561 [1:20:52<00:00,  3.11s/it]


In [16]:
df1 = pd.read_csv("grammar_gpt4o.csv")
df1

,id,original_text,errors,correct_text,0
0,grammar-0,2017年《财富》全球论坛自宣布落户广州以来，《财富》杂志将目光聚焦在广州，以财富为媒，向世...,[],2017年《财富》全球论坛自宣布落户广州以来，《财富》杂志将目光聚焦在广州，以财富为媒，向世...,NaN
1,grammar-1,它渐渐失去了青苍的颜色，变成嫩黄，变成柔绿，枝条变成细瘦，变成娇弱，好像病了的孩子。,[],它渐渐失去了青苍的颜色，变成嫩黄，变成柔绿，枝条变成细瘦，变成娇弱，好像病了的孩子。,NaN
2,grammar-2,可是在从原棉制成纱线的过程，就不像穿着光滑舒适的衣服那样让人愉快了：纱厂工人终日面临着音响，...,"[{'original_error': '音响', 'type': 'wrong_word'...",可是在从原棉制成纱线的过程，就不像穿着光滑舒适的衣服那样让人愉快了：纱厂工人终日面临着噪音，...,NaN
3,grammar-3,热播剧《山海情》的故事发生在宁夏西海固地区。为了保护当地生态环境和群众生活，20世纪八十年代...,[],热播剧《山海情》的故事发生在宁夏西海固地区。为了保护当地生态环境和群众生活，20世纪八十年代...,NaN
4,grammar-4,老同志不但关心我，就是和他一起入伍的新同志也经常帮助我，这使我非常感动。,"[{'original_error': '就是', 'type': 'grammar_err...",老同志不但关心我，而且和他一起入伍的新同志也经常帮助我，这使我非常感动。,NaN
...,...,...,...,...,...
1556,grammar-1556,上海的庙会是流传于中国北方地区的民俗文化活动，是唯一一个中国国家级非物质文化遗产。,[],上海的庙会是流传于中国北方地区的民俗文化活动，是唯一一个中国国家级非物质文化遗产。,NaN
1557,grammar-1557,在这个多姿多彩的世界中，广大芸芸众生共同构成了人类的多样性和繁荣。\n,[],在这个多姿多彩的世界中，广大芸芸众生共同构成了人类的多样性和繁荣。,NaN
1558,grammar-1558,通过学习，使我提高了认识。,"[{'original_error': '使我提高了认识', 'type': 'gramma...",通过学习，我提高了认识。,NaN
1559,grammar-1559,这本小说的情节和文笔可以堪称现代文学的经典之作。\n,[],这本小说的情节和文笔可以堪称现代文学的经典之作。\n,NaN


In [17]:
df1 = df1[['id', 'original_text', 'errors', 'correct_text']]
df1 = df1[df1['original_text'].str.len() > 1]
df1 = df1[df1['correct_text'].str.len() > 1]
df = df1
df

,id,original_text,errors,correct_text
0,grammar-0,2017年《财富》全球论坛自宣布落户广州以来，《财富》杂志将目光聚焦在广州，以财富为媒，向世...,[],2017年《财富》全球论坛自宣布落户广州以来，《财富》杂志将目光聚焦在广州，以财富为媒，向世...
1,grammar-1,它渐渐失去了青苍的颜色，变成嫩黄，变成柔绿，枝条变成细瘦，变成娇弱，好像病了的孩子。,[],它渐渐失去了青苍的颜色，变成嫩黄，变成柔绿，枝条变成细瘦，变成娇弱，好像病了的孩子。
2,grammar-2,可是在从原棉制成纱线的过程，就不像穿着光滑舒适的衣服那样让人愉快了：纱厂工人终日面临着音响，...,"[{'original_error': '音响', 'type': 'wrong_word'...",可是在从原棉制成纱线的过程，就不像穿着光滑舒适的衣服那样让人愉快了：纱厂工人终日面临着噪音，...
3,grammar-3,热播剧《山海情》的故事发生在宁夏西海固地区。为了保护当地生态环境和群众生活，20世纪八十年代...,[],热播剧《山海情》的故事发生在宁夏西海固地区。为了保护当地生态环境和群众生活，20世纪八十年代...
4,grammar-4,老同志不但关心我，就是和他一起入伍的新同志也经常帮助我，这使我非常感动。,"[{'original_error': '就是', 'type': 'grammar_err...",老同志不但关心我，而且和他一起入伍的新同志也经常帮助我，这使我非常感动。
...,...,...,...,...
1556,grammar-1556,上海的庙会是流传于中国北方地区的民俗文化活动，是唯一一个中国国家级非物质文化遗产。,[],上海的庙会是流传于中国北方地区的民俗文化活动，是唯一一个中国国家级非物质文化遗产。
1557,grammar-1557,在这个多姿多彩的世界中，广大芸芸众生共同构成了人类的多样性和繁荣。\n,[],在这个多姿多彩的世界中，广大芸芸众生共同构成了人类的多样性和繁荣。
1558,grammar-1558,通过学习，使我提高了认识。,"[{'original_error': '使我提高了认识', 'type': 'gramma...",通过学习，我提高了认识。
1559,grammar-1559,这本小说的情节和文笔可以堪称现代文学的经典之作。\n,[],这本小说的情节和文笔可以堪称现代文学的经典之作。\n


In [18]:

ds = df.to_dict(orient='records')
print(ds[:3])
# 进行转换
sharegpt_formatted_data = convert_to_sharegpt_format(ds)
sharegpt_formatted_data[:2], len(sharegpt_formatted_data)

[{'id': 'grammar-0', 'original_text': '2017年《财富》全球论坛自宣布落户广州以来，《财富》杂志将目光聚焦在广州，以财富为媒，向世界展示一个充满底蕴又创新进取的广州。', 'errors': '[]', 'correct_text': '2017年《财富》全球论坛自宣布落户广州以来，《财富》杂志将目光聚焦在广州，以财富为媒，向世界展示一个充满底蕴又创新进取的广州。'}, {'id': 'grammar-1', 'original_text': '它渐渐失去了青苍的颜色，变成嫩黄，变成柔绿，枝条变成细瘦，变成娇弱，好像病了的孩子。', 'errors': '[]', 'correct_text': '它渐渐失去了青苍的颜色，变成嫩黄，变成柔绿，枝条变成细瘦，变成娇弱，好像病了的孩子。'}, {'id': 'grammar-2', 'original_text': '可是在从原棉制成纱线的过程，就不像穿着光滑舒适的衣服那样让人愉快了：纱厂工人终日面临着音响，尘埃和湿气的。', 'errors': "[{'original_error': '音响', 'type': 'wrong_word', 'correct': '噪音'}, {'original_error': '的', 'type': 'extra_word', 'correct': ''}]", 'correct_text': '可是在从原棉制成纱线的过程，就不像穿着光滑舒适的衣服那样让人愉快了：纱厂工人终日面临着噪音，尘埃和湿气。'}]


([{'conversations': [[{'from': 'human',
      'value': '文本纠错：\n\n2017年《财富》全球论坛自宣布落户广州以来，《财富》杂志将目光聚焦在广州，以财富为媒，向世界展示一个充满底蕴又创新进取的广州。'},
     {'from': 'gpt',
      'value': '2017年《财富》全球论坛自宣布落户广州以来，《财富》杂志将目光聚焦在广州，以财富为媒，向世界展示一个充满底蕴又创新进取的广州。\n\nerrors:"[]"'}]]},
  {'conversations': [[{'from': 'human',
      'value': '文本纠错：\n\n它渐渐失去了青苍的颜色，变成嫩黄，变成柔绿，枝条变成细瘦，变成娇弱，好像病了的孩子。'},
     {'from': 'gpt',
      'value': '它渐渐失去了青苍的颜色，变成嫩黄，变成柔绿，枝条变成细瘦，变成娇弱，好像病了的孩子。\n\nerrors:"[]"'}]]}],
 1559)

In [19]:
# 输出 JSONL 格式
output_file_path = "sharegpt_formatted_data_grammar.jsonl"

with open(output_file_path, 'w', encoding='utf-8') as f:
    for entry in sharegpt_formatted_data:
        f.write(json.dumps(entry, ensure_ascii=False) + '\n')

print(f"数据已保存到 {output_file_path}")

数据已保存到 sharegpt_formatted_data_grammar.jsonl


## Few shot续写（optional）

In [32]:
def self_instruct_ds(sentence):
    prompt = """
续写中文纠错训练样本集，标准纠错样本格式如下（jsonl格式）：

说明：
- id：样本id，标识而已，可为空。
- original_text，原始文本：输入的原始文本。
- errors，错误列表：使用 errors 数组来列出每个错误，包括原始错误文本片段（original_error）、错误类型（type）和正确的文本（correct）。这有助于模型更精确地理解错误。
- type，错误类型：定义4种错误类型，spelling_error（拼写错误）、grammar_error（语法错误）、missing_word（缺少单词）、extra_word（多余单词）。
- correct_text，正确文本：正确的文本，即纠正后的文本。

要求：
1.只按标准纠错样本格式输出处理后的jsonl结果，不需要解释。
2.请确保输出的jsonl格式正确，以便后续处理。
3.errors是个列表，如果有多个错误，就都写入；如果没有错误，就是空列表。
4.续写5个样本，注意提升样本的多样性、丰富度，续写的样本errors不要为空，错误类型只能属于上面4种错误类型（spelling_error（拼写错误）、grammar_error（语法错误）、missing_word（缺少单词）、extra_word（多余单词））的范围内，可以包括多个错误。

样本：
{"id": "A2-0023-1","original_text": "下个星期，我跟我朋唷打算去法国玩儿。","errors":[{"original_error": "朋唷","type": "spelling_error","correct": "朋友"}],"correct_text": "下个星期，我跟我朋友打算去法国玩儿。"}
{"id": "A2-0029-1","original_text": "对不气，最近我很忙，所以我不会去妳的。","errors":[{"original_error": "不气","type": "spelling_error","correct": "不起"},{"original_error": "妳的","type": "grammar_error","correct": "你的"}],"correct_text": "对不起，最近我很忙，所以我不会去你的。"}
{"id": "A2-0031-1","original_text": "他昨天去了图书馆借了很多书籍。","errors":[{"original_error": "书籍","type": "extra_word","correct": "书"}],"correct_text": "他昨天去了图书馆借了很多书。"}
{"id": "A2-0032-1","original_text": "今天我得去趟超市买牛奶。","errors":[{"original_error": "去趟","type": "missing_word","correct": "去一趟"}],"correct_text": "今天我得去一趟超市买牛奶。"}
{sentence}
    """

    try:
        res = request_gpt4o_azure(prompt.replace("{sentence}", sentence))
    except Exception as e:
        print(e)
        res = ""
    
    # 返回结果，处理
    clean_res = []
    try:
        if res:
            res = res.strip()
            lines = res.split("\n")
            for i in lines:
                if i.startswith("```"):
                    continue
                if i.startswith("{"):
                    d = json.loads(i)
                    clean_res.append(d)
            
    except Exception as e:
        print(e)
    origin_res = res
    return origin_res, clean_res

# 示例问题和回答
q = """
{'id': 'grammar-2', 'original_text': '可是在从原棉制成纱线的过程，就不像穿着光滑舒适的衣服那样让人愉快了：纱厂工人终日面临着音响，尘埃和湿气的。', 'errors': [{'original_error': '音响', 'type': 'word_choice_error', 'correct': '噪音'}, {'original_error': '的', 'type': 'extra_word', 'correct': ''}], 'correct_text': '可是在从原棉制成纱线的过程，就不像穿着光滑舒适的衣服那样让人愉快了：纱厂工人终日面临着噪音，尘埃和湿气。'}
{'id': '4', 'original_text': '老同志不但关心我，就是和他一起入伍的新同志也经常帮助我，这使我非常感动。', 'errors': [{'original_error': '就是', 'type': 'grammar_error', 'correct': '而且'}], 'correct_text': '老同志不但关心我，而且和他一起入伍的新同志也经常帮助我，这使我非常感动。'}
{"id": "-", "original_text": "从争做中国好网民，才年汇聚网络正能量，占领主战场，让各种网络谣言、欺诈、色情信息没有立足之处，从而构建起无限生机与正义的中国网络正能量。", "errors": [{"original_error": "才年", "type": "spelling_error", "correct": "才能"}], "correct_text": "从争做中国好网民，才能汇聚网络正能量，占领主战场，让各种网络谣言、欺诈、色情信息没有立足之处，从而构建起无限生机与正义的中国网络正能量。"}
"""

# 发送请求并获取评估
origin_res, r = self_instruct_ds(q)
origin_res, r

('```jsonl\n{"id": "A2-0040-1","original_text": "昨天晚上我和家人一齐吃了晚饭。","errors":[{"original_error": "一齐","type": "spelling_error","correct": "一起"}],"correct_text": "昨天晚上我和家人一起吃了晚饭。"}\n{"id": "A2-0041-1","original_text": "他对于数学题目完全理解不。","errors":[{"original_error": "不","type": "missing_word","correct": "不了解"}],"correct_text": "他对于数学题目完全理解不了解。"}\n{"id": "A2-0042-1","original_text": "这个小说书封面很好看。","errors":[{"original_error": "小说书","type": "extra_word","correct": "小说"}],"correct_text": "这个小说封面很好看。"}\n{"id": "A2-0043-1","original_text": "明天早上我打算去跑步公园。","errors":[{"original_error": "跑步公园","type": "missing_word","correct": "去公园跑步"}],"correct_text": "明天早上我打算去公园跑步。"}\n{"id": "A2-0044-1","original_text": "我们已经讨论问题了很多次。","errors":[{"original_error": "问题了","type": "grammar_error","correct": "这个问题"}],"correct_text": "我们已经讨论这个问题很多次了。"}\n```',
 [{'id': 'A2-0040-1',
   'original_text': '昨天晚上我和家人一齐吃了晚饭。',
   'errors': [{'original_error': '一齐',
     'type': 'spelling_error',
     'correct': '一起'}],
   'corre